In [4]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [5]:
from crewai import Agent, Crew, Task

In [6]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [7]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [8]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [9]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [10]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

In [11]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [12]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [13]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [14]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

In [15]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

In [21]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "Surat",
    'tentative_date': "2025-04-02",
    'expected_participants': 500,
    'budget': 200000,
    'venue_type': "Conference Hall"
}

In [ ]:
result = event_management_crew.kickoff(inputs=event_details)

In [23]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'Corporate Party Venue',
 'booking_status': 'Rs. 799 per plate',
 'capacity': 50,
 'name': 'Lords Plaza Surat'}


In [24]:
from IPython.display import Markdown
Markdown("marketing_report.md")

# Marketing Activities and Attendee Engagement Report

## Marketing Activities:
1. Utilize Meetup, Facebook groups, Reddit, and advertising campaigns for promotion.
2. Create engaging content and visuals for social media platforms like Facebook, Instagram, and Twitter.
3. Implement email marketing campaigns to reach potential attendees.
4. Partner with sponsors for conference advertising opportunities.
5. Build a strong event website and app for easy registration.
6. Offer incentives to attract more attendees.
7. Analyze data to measure the success of marketing strategies.

## Attendee Engagement Ideas:
1. Include gamification elements like quizzes and trivia games to engage attendees.
2. Organize a trivia competition to help guests relax and interact.
3. Provide speaker orientation sessions for a better attendee experience.
4. Schedule entertainment and fun events throughout the conference.
5. Set up engaging discussion groups to connect attendees.
6. Offer interactive workshops, gamification, and personalized experiences.
7. Engage attendees with live polling and give-back events.
8. Implement innovative activities like scavenger hunts, outdoor yoga, and cooking classes.
9. Explore the outdoors and offer wellness experiences for attendees.
10. Utilize social media, polls, and structured networking sessions to engage attendees effectively.

By implementing these marketing strategies and engagement ideas, we aim to maximize event exposure and attract over 500 potential attendees to the Tech Innovation Conference at Lords Plaza Surat on Ring Road.

Let's make this conference a huge success!